<a href="https://colab.research.google.com/github/Shinyh29/git_collab/blob/main/DeepLOB_printed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
import os
os.chdir("/content/drive/MyDrive/tmf_shinyh")
os.getcwd()

'/content/drive/MyDrive/tmf_shinyh'

In [9]:
## This Notbook runs on TF2
# obtain data
import os 
if not os.path.isfile('data.zip'):
    !wget https://raw.githubusercontent.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/master/data/data.zip
    !unzip -n data.zip
    print('data downloaded.')
else:
    print('data already existed.')

data already existed.


In [10]:
# limit gpu memory
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [11]:
 tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.list_physical_devices('CPU') ## 이제 유료아니면 GPU 연결안해주나봄


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# load packages
import pandas as pd
import pickle
import numpy as np
import keras
from keras import backend as K
from keras.models import load_model, Model
from keras.layers import Flatten, Dense, Dropout, Activation, Input, LSTM, Reshape, Conv2D, MaxPooling2D
#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils

from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
# Data 



def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)
"""
first 40 columns of the FI-2010 dataset 
are 10 levels ask and bid information  // 데이터셋에서  첫 40개 컬럼은  10 level 의  ask, bid 
for a limit order book and we only use these 40 features in our network.  // 40 개 feature  쓸것

"""



def get_label(data):
    lob = data[-5:, :].T
    return lob
    """
     The last 5 columns of the FI-2010 dataset are the labels with different prediction horizons. 
     // 마지막 다섯개 열은  다른 예측범위 . 
    """

def data_classification(X, Y, T):
    [N, D] = X.shape  
    df = np.array(X)
    dY = np.array(Y)
    dataY = dY[T - 1:N]
    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]
    return dataX.reshape(dataX.shape + (1,)), dataY

def prepare_x_y(data, k, T):
    x = prepare_x(data)
    y = get_label(data)
    x, y = data_classification(x, y, T=T)
    y = y[:,k] - 1
    y = np_utils.to_categorical(y, 3)
    return x, y

In [12]:
dec_data = np.loadtxt('Train_Dst_NoAuction_DecPre_CF_7.txt')

In [10]:
print(f'{ dec_data},\n shape : {dec_data.shape}')

[[2.615e-01 2.615e-01 2.614e-01 ... 3.531e-01 3.531e-01 3.532e-01]
 [3.530e-03 2.110e-03 1.220e-03 ... 7.750e-03 7.750e-03 2.000e-03]
 [2.606e-01 2.606e-01 2.606e-01 ... 3.527e-01 3.527e-01 3.527e-01]
 ...
 [2.000e+00 2.000e+00 2.000e+00 ... 2.000e+00 1.000e+00 1.000e+00]
 [2.000e+00 2.000e+00 2.000e+00 ... 2.000e+00 1.000e+00 1.000e+00]
 [2.000e+00 2.000e+00 2.000e+00 ... 1.000e+00 1.000e+00 1.000e+00]],
 shape : (149, 254750)


In [20]:
x = prepare_x(dec_data)
y = get_label(dec_data)
print(x.shape, y.shape)
x, y = data_classification(x, y ,T)
y = y[:,k] - 1
y = np_utils.to_categorical(y, 3)
print(x.shape, y.shape)

(254750, 40) (254750, 5)
(254651, 100, 40, 1) (254651, 3)


In [34]:

"""
k=4 
T= 100
trainX_CNN, trainY_CNN = prepare_x_y(dec_train, k, T)
trainX, trainY = get_x_y(data, k, T)
"""
[N,D] = dec_data[:40,:].T.shape
N, D  


(254750, 40)

In [11]:
print( f"data_getLabel :  {dec_data[-5:,:]} \n\n Label shape : {dec_data[-5:,:].shape}")

data_getLabel :  [[2. 2. 3. ... 2. 2. 1.]
 [2. 2. 3. ... 2. 1. 1.]
 [2. 2. 2. ... 2. 1. 1.]
 [2. 2. 2. ... 2. 1. 1.]
 [2. 2. 2. ... 1. 1. 1.]] 

 Label shape : (5, 254750)


In [16]:
dec_test1 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_7.txt') ## dec_data ==  dec_test1
dec_test2 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_9.txt')

In [13]:
print(f'dec_test1 : {dec_test1}, \n shape {dec_test1.shape}\n')
print('-------------------------------------------------------')
print(f'dec_test2 : {dec_test2}, \n shape {dec_test2.shape}\n')
print('-------------------------------------------------------')
print(f'dec_test3 : {dec_test3}, \n shape {dec_test3.shape}\n')

dec_test1 : [[2.666e-01 2.669e-01 2.665e-01 ... 3.660e-01 3.660e-01 3.660e-01]
 [1.290e-03 3.970e-03 2.290e-03 ... 2.000e-03 2.000e-03 2.000e-03]
 [2.654e-01 2.656e-01 2.654e-01 ... 3.656e-01 3.656e-01 3.656e-01]
 ...
 [2.000e+00 2.000e+00 2.000e+00 ... 1.000e+00 2.000e+00 1.000e+00]
 [2.000e+00 2.000e+00 2.000e+00 ... 1.000e+00 1.000e+00 1.000e+00]
 [2.000e+00 2.000e+00 2.000e+00 ... 1.000e+00 1.000e+00 1.000e+00]], 
 shape (149, 55478)

-------------------------------------------------------
dec_test2 : [[0.268   0.268   0.268   ... 0.3646  0.3646  0.3646 ]
 [0.00299 0.00299 0.00299 ... 0.00289 0.00202 0.00202]
 [0.2674  0.2674  0.2674  ... 0.3641  0.3641  0.3641 ]
 ...
 [2.      2.      2.      ... 2.      2.      2.     ]
 [2.      2.      2.      ... 1.      1.      2.     ]
 [2.      2.      2.      ... 1.      1.      1.     ]], 
 shape (149, 52172)

-------------------------------------------------------
dec_test3 : [[0.268   0.268   0.268   ... 0.3783  0.3783  0.3783 ]
 [0.002

In [17]:
# dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
# dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]
#  m = int(np.floor(dec_data.shape[1] * 0.2))  



dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]  ## 사이즈를 맞추는것
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]  ## 사이즈를 맞추는것
print(f" dec_train.shape {dec_train.shape}")
print(f" dec_val.shape {dec_val.shape}")
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))
print(f" dec_test.shape {dec_test.shape}")
print("--------------------------------")
print( dec_train, dec_val, dec_test )

 dec_train.shape (149, 203800)
 dec_val.shape (149, 50950)
 dec_test.shape (149, 139587)
--------------------------------
[[2.615e-01 2.615e-01 2.614e-01 ... 3.639e-01 3.639e-01 3.639e-01]
 [3.530e-03 2.110e-03 1.220e-03 ... 2.180e-03 2.180e-03 2.180e-03]
 [2.606e-01 2.606e-01 2.606e-01 ... 3.634e-01 3.634e-01 3.634e-01]
 ...
 [2.000e+00 2.000e+00 2.000e+00 ... 2.000e+00 2.000e+00 2.000e+00]
 [2.000e+00 2.000e+00 2.000e+00 ... 2.000e+00 2.000e+00 2.000e+00]
 [2.000e+00 2.000e+00 2.000e+00 ... 2.000e+00 2.000e+00 2.000e+00]] [[0.3639  0.3639  0.3639  ... 0.3531  0.3531  0.3532 ]
 [0.00218 0.00427 0.00427 ... 0.00775 0.00775 0.002  ]
 [0.3634  0.3634  0.3634  ... 0.3527  0.3527  0.3527 ]
 ...
 [2.      2.      2.      ... 2.      1.      1.     ]
 [2.      2.      2.      ... 2.      1.      1.     ]
 [2.      2.      2.      ... 1.      1.      1.     ]] [[2.666e-01 2.669e-01 2.665e-01 ... 3.783e-01 3.783e-01 3.783e-01]
 [1.290e-03 3.970e-03 2.290e-03 ... 1.131e-02 1.131e-02 1.131e-02]


In [14]:
k = 4 # which prediction horizon
T = 100 # the length of a single input
n_hiddens = 64
checkpoint_filepath = os.getcwd() + '/model_tensorflow2/weights'
print(f"{checkpoint_filepath}")


/content/drive/My Drive/tmf_shinyh/model_tensorflow2/weights


In [17]:
## 여기서  colab 은 뻗어버림 ??
## => Colab Pro 대용량 RAM 으로 바꾸면 가능
trainX_CNN, trainY_CNN = prepare_x_y(dec_train, k, T)
valX_CNN, valY_CNN = prepare_x_y(dec_val, k, T)
testX_CNN, testY_CNN = prepare_x_y(dec_test, k, T)

print(trainX_CNN.shape, trainY_CNN.shape)
print(valX_CNN.shape, valY_CNN.shape)
print(testX_CNN.shape, testY_CNN.shape)

(203701, 100, 40, 1) (203701, 3)
(50851, 100, 40, 1) (50851, 3)
(139488, 100, 40, 1) (139488, 3)


In [18]:
print(f"trainX_CNN : {trainX_CNN}")
print(f"-------------------------")
print(f"trainY_CNN : {trainY_CNN}")

print(f"-------------------------")
print(f"-------------------------")

print(f"valX_CNN : {valX_CNN}")
print(f"-------------------------")
print(f"valY_CNN : {valY_CNN}")

print(f"-------------------------")
print(f"-------------------------")

print(f"testX_CNN : {testX_CNN}")
print(f"-------------------------")
print(f"testY_CNN : {testY_CNN}")

print(f"-------------------------")

trainX_CNN : [[[[0.2615 ]
   [0.00353]
   [0.2606 ]
   ...
   [0.00311]
   [0.2579 ]
   [0.00128]]

  [[0.2615 ]
   [0.00211]
   [0.2606 ]
   ...
   [0.00138]
   [0.2588 ]
   [0.00123]]

  [[0.2614 ]
   [0.00122]
   [0.2606 ]
   ...
   [0.00311]
   [0.2588 ]
   [0.00123]]

  ...

  [[0.2626 ]
   [0.00326]
   [0.2617 ]
   ...
   [0.00147]
   [0.2602 ]
   [0.00395]]

  [[0.2626 ]
   [0.002  ]
   [0.2617 ]
   ...
   [0.00138]
   [0.2602 ]
   [0.00395]]

  [[0.2627 ]
   [0.00399]
   [0.2617 ]
   ...
   [0.002  ]
   [0.2602 ]
   [0.00395]]]


 [[[0.2615 ]
   [0.00211]
   [0.2606 ]
   ...
   [0.00138]
   [0.2588 ]
   [0.00123]]

  [[0.2614 ]
   [0.00122]
   [0.2606 ]
   ...
   [0.00311]
   [0.2588 ]
   [0.00123]]

  [[0.2614 ]
   [0.00322]
   [0.2606 ]
   ...
   [0.00138]
   [0.2579 ]
   [0.00128]]

  ...

  [[0.2626 ]
   [0.002  ]
   [0.2617 ]
   ...
   [0.00138]
   [0.2602 ]
   [0.00395]]

  [[0.2627 ]
   [0.00399]
   [0.2617 ]
   ...
   [0.002  ]
   [0.2602 ]
   [0.00395]]

  [[0.2627 ]
 

In [25]:

print(f"np.unique(trainY_CNN) :  {np.unique(trainY_CNN)  }")
print(f"np.unique(valY_CNN) :  {np.unique(valY_CNN)  }")
print(f"np.unique(testY_CNN) :  {np.unique(testY_CNN)  }")

np.unique(trainY_CNN) :  [0. 1.]
np.unique(valY_CNN) :  [0. 1.]
np.unique(testY_CNN) :  [0. 1.]


In [ ]:
## Model Architecture 

def create_deeplob(T, NF, number_of_lstm):
    input_lmd = Input(shape=(T, NF, 1))
    
    # build the convolutional block
    conv_first1 = Conv2D(32, (1, 2), strides=(1, 2))(input_lmd)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = Conv2D(32, (1, 2), strides=(1, 2))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = Conv2D(32, (1, 10))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    
    # build the inception module
    convsecond_1 = Conv2D(64, (1, 1), padding='same')(conv_first1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)
    convsecond_1 = Conv2D(64, (3, 1), padding='same')(convsecond_1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)

    convsecond_2 = Conv2D(64, (1, 1), padding='same')(conv_first1)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)
    convsecond_2 = Conv2D(64, (5, 1), padding='same')(convsecond_2)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)

    convsecond_3 = MaxPooling2D((3, 1), strides=(1, 1), padding='same')(conv_first1)
    convsecond_3 = Conv2D(64, (1, 1), padding='same')(convsecond_3)
    convsecond_3 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_3)
    
    convsecond_output = keras.layers.concatenate([convsecond_1, convsecond_2, convsecond_3], axis=3)
    conv_reshape = Reshape((int(convsecond_output.shape[1]), int(convsecond_output.shape[3])))(convsecond_output)
    conv_reshape = keras.layers.Dropout(0.2, noise_shape=(None, 1, int(conv_reshape.shape[2])))(conv_reshape, training=True)

    # build the last LSTM layer
    conv_lstm = LSTM(number_of_lstm)(conv_reshape)

    # build the output layer
    out = Dense(3, activation='softmax')(conv_lstm)
    model = Model(inputs=input_lmd, outputs=out)
    adam = tf.optimizers.Adam(lr=0.0001) ## from tensorflow.keras.optimizers import Adam
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

deeplob = create_deeplob(trainX_CNN.shape[1], trainX_CNN.shape[2], n_hiddens)
deeplob.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 40, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 20, 32)  96          input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 100, 20, 32)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 100, 20, 32)  4128        leaky_re_lu[0][0]                
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# Train 
## CheckPoint 위치 확인하고 
%%time

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=  checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

deeplob.fit(trainX_CNN, trainY_CNN, validation_data=(valX_CNN, valY_CNN), 
            epochs=200, batch_size=128, verbose=2, callbacks=[model_checkpoint_callback])

Epoch 1/200
1592/1592 - 71s - loss: 1.0179 - accuracy: 0.4250 - val_loss: 1.0878 - val_accuracy: 0.3729
Epoch 2/200
1592/1592 - 65s - loss: 0.9872 - accuracy: 0.4730 - val_loss: 1.0874 - val_accuracy: 0.3737
Epoch 3/200
1592/1592 - 66s - loss: 0.8917 - accuracy: 0.5376 - val_loss: 1.0748 - val_accuracy: 0.4061
Epoch 4/200
1592/1592 - 67s - loss: 0.8482 - accuracy: 0.5727 - val_loss: 1.0666 - val_accuracy: 0.4022
Epoch 5/200
1592/1592 - 67s - loss: 0.8133 - accuracy: 0.6062 - val_loss: 1.0889 - val_accuracy: 0.4041
Epoch 6/200
1592/1592 - 67s - loss: 0.7758 - accuracy: 0.6391 - val_loss: 0.9068 - val_accuracy: 0.5719
Epoch 7/200
1592/1592 - 67s - loss: 0.7408 - accuracy: 0.6678 - val_loss: 0.9766 - val_accuracy: 0.5278
Epoch 8/200
1592/1592 - 67s - loss: 0.7124 - accuracy: 0.6880 - val_loss: 0.8732 - val_accuracy: 0.5937
Epoch 9/200
1592/1592 - 67s - loss: 0.6903 - accuracy: 0.7016 - val_loss: 0.8633 - val_accuracy: 0.6057
Epoch 10/200
1592/1592 - 67s - loss: 0.6716 - accuracy: 0.7137 -

In [ ]:
# Test
deeplob.load_weights(checkpoint_filepath)
pred = deeplob.predict(testX_CNN)
print(pred)

[[0.8887463  0.03698745 0.07426631]
 [0.932749   0.03018244 0.0370685 ]
 [0.9492096  0.02035471 0.03043577]
 ...
 [0.01644406 0.96938163 0.01417422]
 [0.01223178 0.97466063 0.01310756]
 [0.00776428 0.9809404  0.01129529]]


In [ ]:
print(f'testX_CNN.shape: {testX_CNN.shape}')
print(f'pred.shape: {pred.shape}')

testX_CNN.shape: (139488, 100, 40, 1)
pred.shape: (139488, 3)


In [ ]:
print('accuracy_score:', accuracy_score(np.argmax(testY_CNN, axis=1), np.argmax(pred, axis=1)))
print(classification_report(np.argmax(testY_CNN, axis=1), np.argmax(pred, axis=1), digits=4))

accuracy_score: 0.7401353521449874
              precision    recall  f1-score   support

           0     0.7124    0.7486    0.7301     47915
           1     0.8155    0.7272    0.7688     48050
           2     0.7006    0.7451    0.7221     43523

    accuracy                         0.7401    139488
   macro avg     0.7428    0.7403    0.7403    139488
weighted avg     0.7442    0.7401    0.7409    139488



In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(train_losses, label='train loss')
# plt.plot(val_losses, label='validation loss')
# plt.legend()